In [1]:
import time, queue, socket, sqlite3, datetime, threading 
def flight_logger(): 
    con = sqlite3.connect(f'{datetime.datetime.fromtimestamp(time.time()).isoformat()}.db')  
    cur = con.cursor() 
    cur.execute('CREATE TABLE commands(timestamp REAL, command TEXT, who TEXT);')
    cur.execute('CREATE TABLE   states(timestamp REAL, log     TEXT          );') 
    print('Flight Data Record Begins ~') 
    while 1: 
        sql = surrogate.get()
        if sql: 
            cur.execute(sql) 
        else:
            con.commit()
            con.close()  
            print('Flight Data Saved ~') 
            break
    print('Flight Data Record Ends ~')

surrogate = queue.Queue() # cache flight data 
thread = threading.Thread(target=flight_logger)
thread.daemon = True
thread.start()  

class Tello_State:
    def __init__(self):
        self.state  = None 
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        local_ip, state_port = '', 8890 
        self.socket.bind((local_ip, state_port)) 
        self.receive_thread  = threading.Thread(target=self.update_state)
        self.receive_thread.daemon = True
        self.receive_thread.start() 
    
    def update_state(self):
        while True: 
            response, ip = self.socket.recvfrom(1024)    
            surrogate.put('''INSERT INTO states(timestamp, log) 
                                    VALUES({},"{}")
                                 '''.format(time.time(), response))  
            state = response.decode().split(';')
            state.pop() 
            self.state = dict([s.split(':') for s in state]) 
            
state_stat = Tello_State()

class Tello_Command:
    def __init__(self):
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  
        local_ip, receive_port = '', 8889
        self.socket.bind((local_ip, receive_port)) 
        tello_ip, command_port = '192.168.10.1', 8889 
        self.tello_address = (tello_ip, command_port) 
        self.lock = threading.Lock()
        self.lock.acquire()
        self.OK = False
        self.MAX_TIME_OUT = 10
        self.Max_RETRY = 3
        self.receive_thread = threading.Thread(target=self.receive_response)
        self.receive_thread.daemon = True
        self.receive_thread.start() 
        
    def send_command(self, command, debug=False): 
        retry = self.Max_RETRY 
        while True:   
            surrogate.put('''INSERT INTO commands(timestamp, command, who) 
                                    VALUES({}, "{}", "{}");
                                 '''.format(time.time(), command, "Station"))  
            if debug: print('[ Station ] "%s"' % command) 
            self.socket.sendto(command.encode('utf-8'), self.tello_address) 
            retry -= 1
            self.lock.acquire(timeout=self.MAX_TIME_OUT) 
            if self.OK: 
                surrogate.put('''
                    INSERT INTO commands(timestamp, command, who) 
                    VALUES({}, "{}", "{}");
                    '''.format(time.time(), command, "Tello"))  
                if debug: print('[  Tello  ] "%s" ' % command)
                self.OK = False
                break
            else:
                retry -= 1
                if retry < 0:
                    if debug: print('Retried "%s" %d times without a success. Stop to retry' % (command, self.Max_RETRY))
                    break
                if debug: print('Retry "%s"' % command)

    def receive_response(self):
        while True:
            response, ip = self.socket.recvfrom(1024)
            if response == b'ok':
                self.OK = True
                self.lock.release()
            elif response == b'error':
                self.OK = False
                
tello_cmd = Tello_Command()
tello_cmd.send_command('command') 

Flight Data Record Begins ~


In [2]:
print("It may take 10 seconds to take off") 
tello_cmd.send_command('takeoff') 

2020-08-21T09:45:27.616189


'2020-08-21T09:45:36.337272'

In [3]:
tello_cmd.send_command('forward 50')
tello_cmd.send_command('left 50'   )
tello_cmd.send_command('back 50'   )
tello_cmd.send_command('right 50'  , True) 

[ Station ] "right 50"
[  Tello  ] "right 50" 


In [7]:
tello_cmd.send_command('land') 

In [8]:
surrogate.put("") 

Flight Data Saved ~
Flight Data Record Ends ~
